In [5]:
// #r "nuget: NGitLab"
#r "nuget: Fshttp"
#r "nuget: dotenv.net"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"


Installed Packages dotenv.net, 4.0.0 Fshttp, 15.0.3 Plotly.NET, 5.1.0 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\schne\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open System.Text.Json
open dotenv.net

let envvars = DotEnv.Load()

In [3]:
open System.IO

let tree_paths = 
    Directory.GetFiles("data")
    |> Array.ofSeq

In [4]:
open System.Text.Json.Serialization

type RepoFile = {
    id: string
    name: string
    [<JsonPropertyName("type")>]
    ``type``: string
    path: string
    mode: string
}

let jsonTrees =
    tree_paths
    |> Array.map (fun p ->
        p.Split("_")[2],
        File.ReadAllText(p)
        |> Json.JsonSerializer.Deserialize<RepoFile []>
    )

jsonTrees
|> Array.map (fun (id, files) ->
    id, 
    files 
    |> Array.filter (fun f -> 
        f.path.EndsWith(".xlsx")
        && not (f.path.Contains("isa.investigation.xlsx"))
        && not (f.path.Contains("isa.assay.xlsx"))
        && not (f.path.Contains("isa.study.xlsx"))
        && not (f.path.Contains("isa.run.xlsx"))
        && not (f.path.Contains("isa.workflow.xlsx"))
        && not (f.path.Contains("isa.datamap.xlsx"))
    )
)
|> Array.filter (fun (_, files) -> files.Length > 0)
|> Array.map (fun (id, files) ->
    files |> Array.map (fun f -> id, f)
)
|> Array.concat
|> Array.map (fun (id, f) ->
    id, f.path
)
|> DisplayExtensions.DisplayTable

Item1,Item2
1040,assays/MassHunter_targets/dataset/QuantReports/22-0005_exp001.batch_a.bin/190614_QuantReport_ISTD_DB.xlsx
1040,runs/isaSampleToRawDataSeq/rnaseq-samples.xlsx
1048,assays/CMML_15-0001_GCqTOF/protocols/150112_DBrilhaus.xlsx
1048,assays/CMML_15-0001_GCqTOF/protocols/150115_DBrilhaus.xlsx
1048,assays/CMML_15-0001_GCqTOF/protocols/2014-11-27 Dominik HHU_GC-MS_project_info.xlsx
1048,_publication/PP2015-01076R2_Supplemental_Dataset_S1.xlsx
1048,_publication/PP2015-01076R2_Supplemental_Dataset_S2.xlsx
1048,_publication/PP2015-01076R2_Supplemental_Datasets_S3_to_S6.xlsx
1048,assays/CMML_15-0001_GCqTOF-Masshunter-Quant/dataset/150112_Dominik_Brilhaus/QuantReport_ISTD_ResultsComplete_B_06_001_2.xlsx
1048,assays/CMML_15-0001_GCqTOF-Masshunter-Quant/dataset/2015-01-26 Metabolites Summary.xlsx


In [22]:
open Plotly.NET

let interesting_extensions = Set.ofList [
    ".fasta"
    ".fasta.gz"
    ".fastq"
    ".fastq.gz"
    ".fq"
    ".gb"
    ".gbk"
    ".gff"
    ".gff3"
    ".vcf"
    ".sam"
    ".bam"
    ".bed"
    ".wig"
    ".bw"
    ".fa"
    ".fna"
    ".xlsx"
    ".csv"
    ".tsv"
]

let fileExtensions =
    jsonTrees
    |> Array.map snd
    |> Array.concat
    |> Array.map _.path
    |> Array.filter ( fun p ->
        not (p.Contains("isa.investigation.xlsx"))
        && not (p.Contains("isa.assay.xlsx"))
        && not (p.Contains("isa.study.xlsx"))
        && not (p.Contains("isa.run.xlsx"))
        && not (p.Contains("isa.workflow.xlsx"))
        && not (p.Contains("isa.datamap.xlsx")))
    |> Array.map (Path.GetExtension)
    |> Array.filter (fun ext -> interesting_extensions.Contains(ext))
    |> Array.countBy id
    |> Array.sortByDescending snd

fileExtensions
|> Chart.Bar

<!-- Plotly chart will be drawn inside this DIV -->